In [2]:
import os
import cv2   #opencv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from PIL import Image

In [3]:
labels_dict = {
    'Actinic keratosis': 0,
    'Atopic Dermatitis': 1,
    'Benign keratosis': 2,
    'Dermatofibroma': 3,
    'Melanocytic nevus': 4,
    'Melanoma': 5,
    'Tinea Ringworm Candidiasis': 6,
    'Vascular lesion': 7
}

In [4]:
data_dir = '/content/drive/MyDrive/Split_smol/train'

In [5]:
X = []
y = []

for folder, label in labels_dict.items():
    folder_path = os.path.join(data_dir, folder)
    if not os.path.isdir(folder_path):
        print(f"Directory '{folder}' not found in '{data_dir}'. Skipping...")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to read image: {image_path}. Skipping...")
            continue

        img = cv2.resize(img, (227, 227))  # Resize images to (227, 227)
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

In [6]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X_shuffled = X[indices]
y_shuffled = y[indices]   #shuffle to unsure that train and testing data will not follow same order

# Split the data into training and validation sets
split_index = int(0.8 * len(X_shuffled))  # Use 80% of the data for training
X_train, X_val = X_shuffled[:split_index], X_shuffled[split_index:]
y_train, y_val = y_shuffled[:split_index], y_shuffled[split_index:]

In [7]:
datagen = ImageDataGenerator(          #data augumentation
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)



In [8]:
train_generator = datagen.flow(X_train, y_train, batch_size=32)
val_generator = datagen.flow(X_val, y_val, batch_size=32)

print(len(train_generator))
print(len(val_generator))

16
4


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(227, 227, 3))   #googlenet

In [11]:
for layer in base_model.layers:
    layer.trainable = False        #to freeze the data for no longer changes


In [12]:

x = base_model.output
x = GlobalAveragePooling2D()(x)          # x is input
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(labels_dict), activation='softmax')(x)   #softmax - gives probability distribution to determine the class


In [13]:
model = Model(inputs=base_model.input, outputs=output)


In [14]:
from tensorflow.keras.optimizers import Adagrad
optimizer = Adagrad(learning_rate=0.01, initial_accumulator_value=0.1, epsilon=1e-07)

In [15]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [16]:
# from tensorflow.keras.applications import VGG19
# from tensorflow.keras.models import Sequential
# vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(227, 227, 3))
# for layer in vgg19_model.layers:
#     layer.trainable = False

# # Create a new model on top
# with tf.device('/device:GPU:0'):
#     model_vgg19 = Sequential([
#         vgg19_model,
#         GlobalAveragePooling2D(),
#         Dense(512, activation='relu'),
#         Dense(len(labels_dict), activation='softmax')
#     ])

In [ ]:
# model_vgg19.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [17]:
history = model.fit(
    train_generator,
    epochs=100,     #the number of training iterations (epochs) the model will go through.
    validation_data=val_generator
)


Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 2.2224 - accuracy: 0.2069

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-3bc1b4dede38>", line 1, in <cell line: 1>
    history = model.fit(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1841, in fit
    self._eval_data_handler = data_adapter.get_data_handler(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/data_adapter.py", line 1688, in get_data_handler
    return DataHandler(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/data_adapter.py", line 1292, in __init__
    self._adapter = adapter_cls(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/data_adapter.py", line 1024, in __init__
    super()._

TypeError: object of type 'NoneType' has no len()

In [ ]:
test_dir = '/content/drive/MyDrive/Split_smol/val'

In [ ]:
X_test = []
y_test = []

for folder, label in labels_dict.items():
    folder_path = os.path.join(data_dir, folder)
    if not os.path.isdir(folder_path):
        print(f"Directory '{folder}' not found in '{data_dir}'. Skipping...")
        continue

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to read image: {image_path}. Skipping...")
            continue

        img = cv2.resize(img, (227, 227))  # Resize images to (227, 227)
        X_test.append(img)
        y_test.append(label)

X_test = np.array(X)
y_test = np.array(y)

In [ ]:
test_generator = datagen.flow(X_test, y_test, batch_size=32)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Epoch vs Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
model.save('skindisease_model_inceptionv3.h5')

In [ ]:
new_model = tf.keras.models.load_model('skindisease_model_inceptionv3.h5')

In [ ]:
import PIL
image_path = '/content/drive/MyDrive/Split_smol/val/Tinea Ringworm Candidiasis/aug_0_B9780702044496000291_f029-001-9780702044496-1.jpg'
test_image = PIL.Image.open(image_path)
resized_image = test_image.resize((227, 227))
resized_image = np.array(resized_image) / 255.0
resized_image = resized_image[np.newaxis, ...]
prediction = new_model.predict(resized_image)
predicted_class_index = np.argmax(prediction)
print('Predicted class index:', predicted_class_index)

In [ ]:
predicted_label = {v: k for k, v in labels_dict.items()}.get(predicted_class_index)
print('Predicted label:', predicted_label)